In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

In [4]:
customer_info = pd.read_csv("customer_info.csv")

In [5]:
customer_info.head()

,CustomerID,AccountBalance,LastLogin,Name,Age,Address,TransactionID,Amount,SuspiciousFlag
0,1001,9507.272060,2022-01-01,Customer 1001,54,Address 1001,787.0,33.670626,0
1,1002,7408.704536,2022-01-02,Customer 1002,35,Address 1002,NaN,NaN,0
2,1003,1715.321989,2022-01-03,Customer 1003,40,Address 1003,641.0,30.980239,0
3,1004,3101.509134,2022-01-04,Customer 1004,30,Address 1004,450.0,11.167088,0
4,1004,3101.509134,2022-01-04,Customer 1004,30,Address 1004,921.0,45.849784,0


In [6]:
customer_info_clean = customer_info.dropna()

In [7]:
customer_info_clean['LastLogin'] = pd.to_datetime(customer_info_clean['LastLogin'])
customer_info_clean['LastLoginDays'] = (customer_info_clean['LastLogin'] - customer_info_clean['LastLogin'].min()).dt.days
selected_features = ['CustomerID', 'AccountBalance', 'LastLoginDays', 'Age', 'TransactionID', 'Amount']
customer_info_train = customer_info_clean[selected_features]

/var/folders/jj/wdjx0ggx0r7c0nl7jz__x8yc0000gn/T/ipykernel_11216/2623924497.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_info_clean['LastLogin'] = pd.to_datetime(customer_info_clean['LastLogin'])
/var/folders/jj/wdjx0ggx0r7c0nl7jz__x8yc0000gn/T/ipykernel_11216/2623924497.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_info_clean['LastLoginDays'] = (customer_info_clean['LastLogin'] - customer_info_clean['LastLogin'].min()).dt.days


In [8]:
customer_info_train

,CustomerID,AccountBalance,LastLoginDays,Age,TransactionID,Amount
0,1001,9507.272060,0,54,787.0,33.670626
2,1003,1715.321989,2,40,641.0,30.980239
3,1004,3101.509134,3,30,450.0,11.167088
4,1004,3101.509134,3,30,921.0,45.849784
5,1005,5405.766914,4,46,599.0,81.808651
...,...,...,...,...,...,...
1359,1999,7376.854107,998,57,236.0,69.825072
1360,1999,7376.854107,998,57,805.0,34.780068
1361,2000,4853.024380,999,42,44.0,86.971920
1362,2000,4853.024380,999,42,779.0,86.049092


In [9]:
model = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.01)
model.fit(customer_info_train)

IsolationForest(contamination=0.01)

In [10]:
customer_info_clean["Anomaly"] = model.predict(customer_info_train)

/var/folders/jj/wdjx0ggx0r7c0nl7jz__x8yc0000gn/T/ipykernel_11216/2050719866.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_info_clean["Anomaly"] = model.predict(customer_info_train)


In [11]:
customer_info_clean[customer_info_clean["Anomaly"] == -1]

,CustomerID,AccountBalance,LastLogin,Name,Age,Address,TransactionID,Amount,SuspiciousFlag,LastLoginDays,Anomaly
16,1014,9922.291973,2022-01-14,Customer 1014,27,Address 1014,945.0,42.605353,0,13,-1
29,1023,9545.474869,2022-01-23,Customer 1023,34,Address 1023,220.0,95.667155,0,22,-1
37,1027,9527.947107,2022-01-27,Customer 1027,46,Address 1027,2.0,12.881180,0,26,-1
49,1037,9519.477122,2022-02-06,Customer 1037,61,Address 1037,959.0,31.808667,0,36,-1
555,1411,1290.323059,2023-02-15,Customer 1411,19,Address 1411,996.0,19.876750,0,410,-1
1266,1928,9352.655675,2024-07-16,Customer 1928,60,Address 1928,999.0,93.633040,0,927,-1
1282,1942,9839.150446,2024-07-30,Customer 1942,51,Address 1942,954.0,98.300410,0,941,-1
1295,1951,6303.229480,2024-08-08,Customer 1951,18,Address 1951,970.0,96.109351,0,950,-1
1350,1994,8791.304689,2024-09-20,Customer 1994,63,Address 1994,48.0,17.640569,0,993,-1
1358,1998,9148.652759,2024-09-24,Customer 1998,25,Address 1998,152.0,10.642141,0,997,-1


In [12]:
import joblib
with open("model.joblib", "rb") as file:
    model = joblib.load(file)

In [14]:
model.__dict__

{'estimator': ExtraTreeRegressor(max_depth=8, max_features=1),
 'n_estimators': 100,
 'estimator_params': (),
 'base_estimator': 'deprecated',
 'max_samples': 'auto',
 'max_features': 1.0,
 'bootstrap': False,
 'bootstrap_features': False,
 'oob_score': False,
 'warm_start': False,
 'n_jobs': None,
 'random_state': None,
 'verbose': 0,
 'contamination': 0.01,
 'feature_names_in_': array(['CustomerID', 'AccountBalance', 'LastLoginDays', 'Age',
        'TransactionID', 'Amount'], dtype=object),
 'n_features_in_': 6,
 'max_samples_': 256,
 '_n_samples': 1000,
 'estimator_': ExtraTreeRegressor(max_depth=8, max_features=1),
 '_max_samples': 256,
 '_max_features': 6,
 'estimators_': [ExtraTreeRegressor(max_depth=8, max_features=1, random_state=1909497914),
  ExtraTreeRegressor(max_depth=8, max_features=1, random_state=1354318365),
  ExtraTreeRegressor(max_depth=8, max_features=1, random_state=256289809),
  ExtraTreeRegressor(max_depth=8, max_features=1, random_state=905291948),
  ExtraTreeRe